In [ ]:
#model1 
import argparse
import torch
import pandas as pd
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import BertConfig, BertForSequenceClassification, AdamW, get_scheduler
# from datasets import load_metric
from transformers import TrainingArguments
# from transformers import Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.datasets import load_svmlight_file

In [ ]:
dff1 =  pd.read_csv("Female_aligned_data.csv")
dff2 =  pd.read_csv("Male_aligned_data.csv")

In [ ]:
df = pd.concat([dff1, dff2], ignore_index=True)

In [ ]:
import re 
clean_txt = []
for w in range(len(df['0'])):
    desc = df['0'][w].lower()
    desc = re.sub('[^a-zA-Z]', ' ', desc) # punc
    desc=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",desc) #tags
    desc=re.sub("(\\d|\\W)+"," ",desc)
    clean_txt.append(desc)

In [ ]:
df['clean'] = clean_txt

In [ ]:
sentences = df['clean']
labels_train = 0

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer, BertModel


tokenizer = BertTokenizer.from_pretrained("Models/")
model = BertMLM.from_pretrained("Models/")

In [ ]:
input_ids_train = []
attention_masks_train = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                    
                        add_special_tokens = True,
                        max_length = 512,           
                        pad_to_max_length = True,
                        return_attention_mask = True,  
                        return_tensors = 'pt',     
                        truncation=True
                   )
    
    input_ids_train.append(encoded_dict['input_ids'])
    
    attention_masks_train.append(encoded_dict['attention_mask'])

input_ids_train = torch.cat(input_ids_train, dim=0)
attention_masks_train = torch.cat(attention_masks_train, dim=0)
labels_train = torch.tensor(labels_train)

In [ ]:
from torch.utils.data import TensorDataset, random_split

train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)

In [ ]:
train_size = len(train_dataset)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 4


train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size,
            
        )

In [ ]:
# get sentence embeddings 

In [ ]:
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

In [ ]:
# h1.remove()
h1 = model.encoder.layer[11].output.dropout.register_forward_hook(get_features('feats'))
# h1 = model.bert.pooler.register_forward_hook(get_features('feats'))
# h1 = model.bert.embeddings.register_forward_hook(get_features('feats'))

In [ ]:
device = 'cuda:0'

In [ ]:
model.to(device)

In [ ]:
PREDS = []
FEATS = []
cnt = 0
# placeholder for batch features
features = {}
k = 0
# loop through batches

for step, batch in enumerate(train_dataloader):
    input_ids = batch[0].to(device, dtype=torch.long)
    mask = batch[1].to(device, dtype=torch.long)
    labels = batch[2].to(device, dtype=torch.long)
    print(cnt)

    preds = model(input_ids=input_ids, attention_mask=mask)
    FEATS.append(features['feats'].cpu().numpy())
    cnt = cnt + 1 

In [ ]:
import numpy as np
FEATS_encoder_0 = np.concatenate(FEATS)

In [ ]:
print('- feats shape:', FEATS_encoder_0.shape)
feats_enc = np.mean(FEATS_encoder_0, axis = (1)) 

In [ ]:
pd.DataFrame(feats_enc).to_csv("embeds_1.csv")